## Lung Segmentation using UNets

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import os, os.path as osp

import imgaug.augmenters as iaa

from deeply.model.unet import (
    UNet,
    Trainer
)
from deeply.plots import segplot, history_plot
from deeply.datasets.util import split as split_dataset
from deeply.generators import ImageMaskGenerator

from deeply.util.system import makedirs

from deeply.util.image  import augment as augment_images

from deeply._compat import iteritems

import deeply.datasets as dd

import tqdm.notebook as tq

#### Constants

In [ ]:
image_size    = (256, 256)
width, height = image_size
batch_size = 1

split_types = ("train", "val", "test")

#### Build Model

In [ ]:
unet = UNet(x = width, y = height, n_classes = 1, final_activation = "sigmoid",
            batch_norm = False, padding = "same")

In [ ]:
unet.plot()

In [ ]:
output_shape = unet.output_shape[1:-1]

#### Prepare Dataset

In [ ]:
!rm -rf ./data

In [ ]:
names   = ("montgomery",)

image_augmentor = iaa.Sequential([
    iaa.Resize({ "width": width, "height": height }),
    iaa.CLAHE(
        from_colorspace = iaa.CLAHE.RGB,
        to_colorspace   = iaa.CLAHE.HSV
    )
])

mask_augmentor  = iaa.Sequential([
    iaa.Resize({ "width": output_shape[0], "height": output_shape[1] })
])

for name in names:
    dataset = split_dataset(dd.load(name, shuffle_files = True)["data"])
    groups  = dict(zip(split_types, dataset))
    
    for split_type, split in iteritems(groups):
        print("Generating %s images..." % split_type)
        
        dir_name = osp.join("data", split_type)
        makedirs(dir_name, exist_ok = True)

        for i, data in enumerate(tq.tqdm(split.batch(1))):
            augment_images(image_augmentor, images = data["image"].numpy(), filename = osp.join(dir_name, "images", "%s.jpg" % i))
            augment_images(mask_augmentor,  images = data["mask"].numpy(),  filename = osp.join(dir_name, "masks",  "%s.jpg" % i))

In [ ]:
args = dict(batch_size = batch_size, color_mode = "grayscale", image_size = image_size, mask_size = output_shape, shuffle = True)
train, val, test = [ImageMaskGenerator("data/%s/images" % type_, "data/%s/masks" % type_, **args) for type_ in split_types]

#### Show Samples

In [ ]:
from itertools import islice

In [ ]:
n_samples = 3

In [ ]:
samples = islice(test, n_samples)

In [ ]:
for image, mask in samples:
    segplot(image[0], mask[0])

In [ ]:
from tensorflow.keras.optimizers import Adam

from deeply.losses import dice_loss

In [ ]:
unet.compile(optimizer = Adam(learning_rate = 1e-5), loss = dice_loss, metrics = ["binary_accuracy"])

In [ ]:
epochs = 10

#### Preprocess Data

In [ ]:
trainer = Trainer()
history = trainer.fit(unet, train, val = val, epochs = epochs)

In [ ]:
history_plot(history)

In [ ]:
predict = unet.predict(test)